In [2]:
import os 
import time 
import json,math,sys,re
import pandas as pd 

In [ ]:
folder = "../.."
from dotenv import load_dotenv
env_path = f'{folder}/deepseek.env'
load_dotenv(dotenv_path=env_path)
DEEPSEEK_KEY = os.getenv('DEEPSEEK_KEY')
from openai import OpenAI
client = OpenAI(api_key=DEEPSEEK_KEY, base_url="https://api.deepseek.com")

In [ ]:
fname = "parallel_predictions_SAMPLE - FALSE"
parallel = pd.read_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/{fname}.csv").fillna('')
parallel = parallel.to_dict(orient='records')
parallel[0], len(parallel)

In [ ]:
fname = "parallel_verses_UNCONFIDENT - HNDPR"
parallel = pd.read_csv(f"{folder}/QP/{fname}.csv").fillna('')
parallel = parallel.to_dict(orient='records')
for item in parallel: 
    item['text'] = item['ESV'] + " " + item['text']
parallel[0], len(parallel)

In [ ]:
prompt = """You are given pairs of Bible verses from different Bible versions. 
            Verse 1 is always from the English Standard Version. Verse 2 comes from a different version. 
            As an Early Modern English and Latin expert, you determine whether they have similar meanings or very similar language. 
            If Verse 2 is in Latin, compare its translation with Verse 1 to see if it refers to the same idea, even if the numbering might differ. 
            For example, "Psalms 113.2 facta est Judaea sanctificatio ejus; Israel potestas ejus." from the Vulgate is equivalent to "Psalms 114.2 Judah became his sanctuary, Israel his dominion." from the ESV.
            Return a True/False answer without giving any of your reasoning. 
        """

You are given pairs of Bible verses from different Bible versions. 
Verse 1 is always from the English Standard Version. Verse 2 comes from a different version. 
As an Early Modern English and Latin expert, you determine whether they have similar meanings or very similar language. 
If Verse 2 is in Latin, compare its translation with Verse 1 to see if it refers to the same idea, even if the numbering might differ. 
For example, "Psalms 113.2 facta est Judaea sanctificatio ejus; Israel potestas ejus." from the Vulgate is equivalent to "Psalms 114.2 Judah became his sanctuary, Israel his dominion." from the ESV.
Return a True/False answer without giving any of your reasoning. 

Input Verse 1 from the ESV: "Genesis 4.21 His brother's name was Jubal; he was the father of all those who play the lyre and pipe."
Input Verse 2 from the Wycliffe version: 'Genesis 4.21 and the name of his brother was Tubal, he was the fadir of syngeris in harpe and orgun.'

Output:


In [ ]:
with open(f"{folder}/QP/predictions/{fname}_DSV3.json",'r') as f: 
    responses = json.load(f) 
model = "V3" 

In [ ]:
from tqdm import tqdm 

for idx, item in enumerate(tqdm(parallel)):
    key = "{}; {} ({})".format(item['ESV'], item['Parallel'], item['version'])
    if key in responses: continue 

    response = client.chat.completions.create(
        model='deepseek-chat',
        max_tokens = 2, 
        messages=[ 
            {"role": "user",  
             "content": f"{prompt}\n\nInput Verse 1 from the ESV: {item['text']}\nInput Verse 2 from the {item['version']}: {item['verse_text']}\n\nOutput:"
            }
        ],
        stream=False
    )
    
    responses[key] = response.choices[0].message.content
    if (idx+1) % 500 == 0: 
        with open(f"{folder}/QP/predictions/{fname}_DS{model}.json",'w') as f: 
            json.dump(responses,f) 

with open(f"{folder}/QP/predictions/{fname}_DS{model}.json",'w') as f: 
    json.dump(responses,f) 
    # 100%|██████████| 4380/4380 [4:56:13<00:00,  4.06s/it] 
    # 100%|██████████| 3291/3291 [3:09:16<00:00,  3.45s/it]

In [ ]:
output = []
found = {}
for item in parallel:
  key = "{}; {} ({})".format(item['ESV'], item['Parallel'], item['version'])
  if key in responses: 
    key2 = (item['ESV'],item['version'])
    item['prediction'] = responses[key].capitalize()
    if item['prediction'] == 'True': 
      found[key2] = True 
    output.append(item)
  else: 
    item['prediction'] = None 
    output.append(item)

for item in parallel: 
  key2 = (item['ESV'],item['version'])
  if key2 in found: 
    item['toCheck'] = False
  else: 
    item['toCheck'] = True 
output = pd.DataFrame(output)
output = output.sort_values(by='ESV')
output['correction'] = ''
print(len(output[output['toCheck'] == True]))
output.to_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/parallel_verses_UNCONFIDENT - LABELED.csv",index=False)
output

In [ ]:
output = []
preds = responses
for idx, entry in enumerate(parallel):
  if idx in preds:
    entry['prediction'] = preds[idx].capitalize()
  output.append(entry)

items = pd.read_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/parallel_predictions_SAMPLE - FALSE.csv").fillna('').to_dict(orient='records')
output.extend(items) 
output = pd.DataFrame(output)
output = output.sort_values(by="index")
output.to_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/parallel_predictions_SAMPLE - FALSE.csv",index=False)
output

In [ ]:
output.to_csv(f"{folder}/Early-Modern-Sermons/assets/QP_Datasets/parallel_predictions_SAMPLE - FALSE.csv",index=False)


In [ ]:
os.remove("../assets/QP_Datasets/parallel_predictions_SAMPLE - NEW.csv")